# TN Districts

## Progress

The Goobers have scraped the website

## The Imports

In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
from datetime import datetime
from lxml import html
import requests
import re

## The Soup Prep

In [11]:
def get_district (State, District):
    url = "https://www.opensecrets.org/races/candidates?cycle=2020&id="+State+District+"&spec=N"
    page = requests.get(url)
    soup = BS(page.content, 'html.parser')
    #Demographic information soup.
    persons = soup.findAll('div',  {'class':"Members--list-item"})
    #Funds raised and spent soup.
    money = soup.find_all('td', {'class': "Members--number"}) 
    #Loops for name, party, votes, state, etc.
    for person in persons:
        name_list.append(get_name (person))
        party_list.append(get_party (person))
        votes_received.append(votes (person))
        state_list.append(State)
        district_list.append(District)
        winner_list.append(winner(person))
        incumbent_list.append(incumbent(person))
    #Loop for finding amount of money raised
    starting_index = 0
    end = len(money) //3 -1
    raised.append(money[starting_index].text)
    for index in range(end):
        starting_index = starting_index + 3
        raised.append(money[starting_index].text)
    starting_index = 1
    end = len(money) //3 -1
    spent.append(money[starting_index].text)
    for index in range(end):
        starting_index = starting_index + 3
        spent.append(money[starting_index].text)

## The Soup Functions

In [3]:
#Function to clean name and append to list.
def get_name (person):
    member_text = person.find("h2").text
    member_text = member_text.replace("\t", "").replace("\n", "")
    return re.search(r"(.+) \(.\)", member_text)[1]
#Function to clean affiliation and append to list
def get_party (person):
    member_text = person.find("h2").text
    member_text = member_text.replace("\t", "").replace("\n", "")
    return re.search(r"\((.)\)", member_text)[1]
#Function clean votes and append to list.
def votes (person):
    member_text = person.find("h2").text
    member_text = member_text.replace("\t", "").replace("\n", "")
    return re.search("\((\d+\.\d)\%", member_text)[1]
#Function to find money raised, spent append to list
def get_money(amount):
    money_text = amount.text
    return re.search(r"\d+.\d{3}.\d{3}", money_text)[0]
def winner(person):
#Function finds if candidate is winner and append to list.   
    if 'Winner' in [x.text for x in person.find_all('i')]:
        return(True)
    else:
        return(False)
#Function finds if canditdate was incumbant and append to list.
def incumbent(person):
    if 'Incumbent' in [x.text for x in person.find_all('i')]:
        return(True)
    else:
        return(False)

## The List Loops

## The Master Lists

In [96]:
#Lists populated by the soup functions
name_list = []
party_list = []
state_list = []
district_list = []
votes_received = []
raised = []
spent = []
winner_list = []
incumbent_list = []
############################################
#Pull this lever to feed soup to the panda:#
############################################
State = 'Sc' #Two letter state code
for district in range(1, 3): #Districts in state
    #converts numbers from range to strings
    get_district(State, str(district).zfill(2))

## The Pandas Food
### (╯°□°)╯︵ ┻━┻

In [97]:
#Dataframe constructed from lists and functions above.
#Saves info from each state to its own local .csv.
Shoots_and_Leaves = pd.DataFrame({"Name": name_list,
              "Party": party_list, 
              "Percent_Vote": votes_received, 
              "Amount_Raised" : raised, 
              "Amount_Spent"  : spent,
              "State" : state_list,
              "District" : district_list,
              "Winner" : winner_list,
              "Incumbent" : incumbent_list 
             })
Shoots_and_Leaves.to_csv(f'../Data/AllStates/{State}.csv', index=False) 

# An Analysis of Political Contributions During the 2020 House of Representatives Election

In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/.

1. Start by scraping the data from the summary page for Tennessee's 7th District, which is available at https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N.
   

* Make a DataFrame showing, for each candidate:
    * the candidate's name
    * the candidate's party
    * state
    * district number
    * whether the candidate was an incumbent
    * whether the candidate won the race
    * the percentage of the vote that candidate received
    * the total amount raised by that candidate (as a numeric variable)
    * the total amount spent by the candidate (as a numeric variable)

2. Once you have working code for Tennessee's 7th District, expand on your code to capture all of Tennessee's districts.

3. Once you have working code for all of Tennessee's districts, expand on it to capture all states and districts. The number of representatives each state has can be found in a table on this page: https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120

4. Using your scraped data, investigates different relationships between candidates and the amount of money they raised. Here are some suggestions to get you started, but feel free to pose you own questions or do additional exploration:  
    a. How often does the candidate who raised more money win a race?  
    b. How often does the candidate who spent more money win a race?  
    c. Does the difference between either money raised or money spent seem to influence the likelihood of a candidate winning a race?  
    d. How often does the incumbent candidate win a race?  
    e. Can you detect any relationship between amount of money raised and the incumbent status of a candidate?

### Bonus Questions:
If you complete all of the above, you can attempt these challenging bonus questions.

Open Secrets also gives a detailed breakdown of contributions by source.

Scrape these pages to get information on contributions by source. See if you can find anything interesting in terms of the source of contributions. Some examples to get you started:
* What does the overall distribution of funding sources look like?
* Is there any detectable difference in contribution sources between Democrat and Republican candidates?
* Do the funding sources for either the winning candidate or incumbent candidate differ from the other candidates?